## Prediction with CART – data preparation
Case studies:                     
  - CH15A Predicting used car value with regression trees 
                                   
Dataset:

    used-cars

In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
from skimpy import skim

warnings.filterwarnings("ignore")

Import data

In [ ]:
data = pd.read_csv("https://osf.io/7gvz9/download")

In [ ]:
data.head()

In [ ]:
data.shape

### Sample design

Manage missing

In [ ]:
data["fuel"] = data["fuel"].fillna("Missing")
data["drive"] = data["drive"].fillna("Missing")
data["cylinders"] = data["cylinders"].fillna("Missing")
data["transmission"] = data["transmission"].fillna("Missing")
data["type"] = data["type"].fillna("Missing")

Missing changed to good not missing for condition

In [ ]:
data["condition"].value_counts()


In [ ]:
data["condition"] = data["condition"].fillna("good")


In [ ]:
data["condition"].value_counts()


Drop hybrid models then drop column

In [ ]:
data = data.loc[lambda x: x["Hybrid"] == 0].drop("Hybrid", axis=1)

Keep gas-fuelled vehicles

In [ ]:
data = data.loc[lambda x: x["fuel"] == "gas"]

Drop vehicles in fair and new condition, trucks

In [ ]:
data = data.loc[lambda x: ~x["condition"].isin(["new","fair"])]

Drop unrealistic values for price and odometer reading


In [ ]:
data = data.loc[lambda x: (x["price"].isin(range(500, 25001))) & (x["odometer"] <= 100)]

Drop if price is smaller than 1000 and condition is like new or age is less than 8


In [ ]:
data = data.loc[
    lambda x: ~((x["price"] < 1000) & ((x["condition"] == "like new") | (x["age"] < 8)))
]

In [ ]:
data = data.loc[lambda x: x["transmission"] != "manual"]

Drop if truck

In [ ]:
data = data.loc[lambda x: ~x["type"].isin(["truck", "pickup"])]

Drop price string

In [ ]:
data = data.drop("pricestr",axis=1)

To be on the safe side, drop NA prices

In [ ]:
data = data.loc[lambda x: x["price"].notna()]

### Data generation & descriptives

Variables we are interested in:
   
       price age odometer + condition cylinder dealer city LE

Condition

In [ ]:
data["cond_excellent"] = np.where(data["condition"] == "excellent", 1, 0)
data["cond_good"] = np.where(data["condition"] == "good", 1, 0)
data["cond_likenew"] = np.where(data["condition"] == "like new", 1, 0)

Cylinders

In [ ]:
data.cylinders.value_counts()

In [ ]:
data["cylind6"] = np.where(data["cylinders"] == "6 cylinders", 1, 0)
data.cylind6.value_counts()

Chicago


In [ ]:
data["chicago"] = np.where(data["area"] == "chicago", 1, 0)

age: quadratic, cubic

In [ ]:
data["agesq"] = data["age"] ** 2
data["agecu"] = data["age"] ** 3

odometer quadratic

In [ ]:
data["odometersq"] = data["odometer"] ** 3

Take a look at descrpitives

In [ ]:
skim(data)

In [ ]:
data["price"].describe()

In [ ]:
data["price"].hist()

In [ ]:
data["price"].apply(np.log).hist()

Save data for prediction

In [ ]:
os.makedirs("data", exist_ok=True)
data.to_csv("data/usedcars_cart_work.csv", index=False)